<a href="https://colab.research.google.com/github/lanej5/mlb/blob/main/elo/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate features

The primary output of this notebook is `features.parquet`.

Note that `features.parquet` contains *two* label columns:
- `RD` - the run differential of a game (scalar label)
- `W` - a binary label (1 if home team won, 0 if visitors won)

Note that tie games are dropped (there are only 13 in the date range we consider).

### todo

features to add:
- number days rest
- average number days rest in past period
- other defensive stats
- sabr stuff

other things to try:
- different types of window

## References:
- https://www.mlb.com/stats/team/batting-average
- https://www.mlb.com/stats/team/pitching?sortState=asc

In [112]:
import pandas as pd
pd.options.display.max_columns = None

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
path = '/content/drive/My Drive/mlb/retrosheet/'
! ls '/content/drive/My Drive/mlb/retrosheet/'

biofile.csv  GL1995.csv  GL2001.csv  GL2007.csv  GL2013.csv  GL2019.csv
GL1990.csv   GL1996.csv  GL2002.csv  GL2008.csv  GL2014.csv  GL2020.csv
GL1991.csv   GL1997.csv  GL2003.csv  GL2009.csv  GL2015.csv  GL2021.csv
GL1992.csv   GL1998.csv  GL2004.csv  GL2010.csv  GL2016.csv  LEAGUEABR.csv
GL1993.csv   GL1999.csv  GL2005.csv  GL2011.csv  GL2017.csv  parks.csv
GL1994.csv   GL2000.csv  GL2006.csv  GL2012.csv  GL2018.csv  TEAMABR.csv


# legend

In [115]:
!wget https://www.retrosheet.org/gamelogs/glfields.txt

!cat glfields.txt

--2022-06-16 18:30:04--  https://www.retrosheet.org/gamelogs/glfields.txt
Resolving www.retrosheet.org (www.retrosheet.org)... 192.124.249.9
Connecting to www.retrosheet.org (www.retrosheet.org)|192.124.249.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6559 (6.4K) [text/plain]
Saving to: ‘glfields.txt’

glfields.txt        100%[===================>]   6.41K  --.-KB/s    in 0s      

2022-06-16 18:30:05 (231 MB/s) - ‘glfields.txt’ saved [6559/6559]

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B"

# load, clean, concat game logs 1990 - 2021

In [123]:
import pandas as pd

def get_team_index_map(file):
    """Returns a dict mapping team abbr from 1990 - 2021 to a
    dense set of indices.

    If a team changed at some point between 1990 - 2021, the old
    team abbr is mapped to the same index as the new team abbr.

    Args:
    -----
    file: valid path to TEAMABR.csv from retrosheets.
    """
    teams = pd.read_csv(file)

    # filter out teams that ceased to exist before 2021
    current_teams = teams[teams['Last year of this combination'] == 2021] \
    .reset_index(drop=True)

    # create mapping
    team_index_map = dict(zip(current_teams['Team Abbreviation'], current_teams.index))

    # add changed team abbr
    team_index_map['MON'] = team_index_map['WAS']
    team_index_map['CAL'] = team_index_map['ANA']
    team_index_map['FLO'] = team_index_map['MIA']

    # save as a dataframe
    team_ids = pd.DataFrame(team_id_map.items(), columns=['abbr', 'id'])
    team_ids.to_csv('/content/drive/My Drive/mlb/features/team_ids.csv')

    return team_index_map

def clean(gl):
    """
    clean the dataframe:
        1. parse dates
        2. convert abbr to team ids 
        3. rename stat columns 
        4. add run differential and win columns
        5. drop tie games
        6. add integer date index 
        7. sort dates
        8. add game index
    """
    team_ids = get_team_index_map('/content/drive/My Drive/mlb/retrosheet/TEAMABR.csv')

    # dict for renaming stat columns
    cols = {
        '4': 'visitor_L',
        '7': 'home_L',
        '9': 'visitor_R',
        '10': 'home_R',
        '21': 'visitor_AB',
        '22': 'visitor_H',
        '23': 'visitor_2B',
        '24': 'visitor_3B',
        '25': 'visitor_HR',
        '28': 'visitor_SF',
        '29': 'visitor_HBP',
        '30': 'visitor_BB',
        '40': 'visitor_ER',
        '49': 'home_AB',
        '50': 'home_H',
        '51': 'home_2B',
        '52': 'home_3B',
        '53': 'home_HR',
        '56': 'home_SF',
        '57': 'home_HBP',
        '58': 'home_BB',
        '68': 'home_ER'
    }

    # parse dates
    gl = gl.assign(  
        date=pd.to_datetime(gl['0'], format='%Y%m%d') \
            .apply(lambda x: x.date())
    )

    # replace abbr with team_id
    gl = gl.assign(visitor_id=gl['3'].map(team_id_map)) \
    .assign(home_id=gl['6'].map(team_id_map))

    # rename columns
    gl = gl.rename(columns=cols)

    # drop unnamed columns
    to_drop = list(set(gl.columns).intersection([str(i) for i in range(161)]))
    gl = gl.drop(columns=to_drop)
    
    # assign
    gl = gl.assign(home_1B=gl['home_H'] - gl['home_2B'] - gl['home_3B'] - gl['home_HR'])
    gl = gl.assign(visitor_1B=gl['visitor_H'] - gl['visitor_2B'] - gl['visitor_3B'] - gl['visitor_HR'])
    gl = gl.assign(RD=gl['home_R'] - gl['visitor_R'])
    gl = gl.assign(W=(gl['RD'] > 0).astype(int))
    gl = gl.assign(interleague=(gl['home_L'] != gl['visitor_L']).astype(int))

    # drop tie games
    gl = gl.drop(gl[gl['RD'] == 0].index)

    # add date and game indexes
    gl = gl.assign(date_index=gl['date'].rank(method='dense'))
    gl = gl.sort_values('date')
    gl = gl.reset_index(drop=True) \
    .reset_index(drop=False).rename(columns={'index': 'game_id'})

    return gl

def load_game_logs():
    gls = []

    for y in range(1990, 2022):
        filename = '/content/drive/My Drive/mlb/retrosheet/GL' + str(y) + '.csv'
        gls.append(pd.read_csv(filename))

    return pd.concat(gls).pipe(clean)

In [124]:
# create game logs
gl = load_game_logs()
gl.head()

,game_id,visitor_L,home_L,visitor_R,home_R,visitor_AB,visitor_H,visitor_2B,visitor_3B,visitor_HR,visitor_SF,visitor_HBP,visitor_BB,visitor_ER,home_AB,home_H,home_2B,home_3B,home_HR,home_SF,home_HBP,home_BB,home_ER,date,visitor_id,home_id,home_1B,visitor_1B,RD,W,interleague,date_index
0,0,AL,AL,2,5,33,5,1,0,0,1,0,4,2,33,8,4,0,0,0,0,4,2,1990-04-09,5,2,4,4,3,1,0,1.0
1,1,AL,AL,7,4,40,15,2,0,1,0,0,1,2,31,5,0,0,1,1,0,5,7,1990-04-09,11,0,4,12,-3,0,0,1.0
2,2,AL,AL,1,2,30,4,1,0,0,0,1,1,2,29,6,1,1,0,1,0,2,1,1990-04-09,23,3,4,3,1,1,0,1.0
3,3,AL,AL,3,8,37,10,1,0,1,0,0,2,7,32,11,2,0,2,1,0,6,3,1990-04-09,8,10,7,8,5,1,0,1.0
4,4,AL,AL,2,4,28,3,2,0,0,0,0,5,4,30,7,1,1,2,0,1,6,2,1990-04-09,14,13,3,1,2,1,0,1.0


# create team log

In [125]:
def get_team_logs(gl):
    # create team log

    # columns to use
    cols = [
        'game_id',
        'RD',
        'W'
    ] + [c for c in gl.columns if c.startswith(('home', 'visitor'))]

    # home logs
    h_rename = {
        **{c: 'team' + c[4:] for c in gl.columns if c.startswith('home')},
        **{c: 'opp' + c[7:] for c in gl.columns if c.startswith('visitor')}
    }

    hl = gl[cols].rename(columns=h_rename)

    # visitor logs
    v_rename = {
        **{c: 'opp' + c[4:] for c in gl.columns if c.startswith('home')},
        **{c: 'team' + c[7:] for c in gl.columns if c.startswith('visitor')}
    }

    vl = gl[cols].rename(columns=v_rename)

    vl['RD'] = - vl['RD']
    vl['W'] = 1 - vl['W']

    tl = pd.concat([hl, vl], axis=0) \
        .reset_index(drop=True) \
        .sort_values(['team_id', 'game_id'])

    return tl

tl = get_team_logs(gl)
tl.head(15)

,game_id,RD,W,opp_L,team_L,opp_R,team_R,opp_AB,opp_H,opp_2B,opp_3B,opp_HR,opp_SF,opp_HBP,opp_BB,opp_ER,team_AB,team_H,team_2B,team_3B,team_HR,team_SF,team_HBP,team_BB,team_ER,opp_id,team_id,team_1B,opp_1B
1,1,-3,0,AL,AL,7,4,40,15,2,0,1,0,0,1,2,31,5,0,0,1,1,0,5,7,11,0,4,12
15,15,7,1,AL,AL,0,7,30,4,2,0,0,0,0,1,7,36,12,1,0,3,1,0,4,0,11,0,8,2
25,25,1,1,AL,AL,0,1,25,0,0,0,0,0,0,4,1,30,7,3,0,0,0,0,6,0,11,0,4,0
35,35,2,1,AL,AL,1,3,29,6,1,0,0,1,0,6,0,30,6,1,0,0,0,0,0,1,8,0,5,5
47,47,-3,0,AL,AL,7,4,33,11,3,0,0,2,1,6,3,35,9,2,0,1,0,0,2,2,8,0,6,8
58,58,2,1,AL,AL,5,7,45,12,3,1,1,0,0,5,7,44,11,3,0,3,0,0,5,4,8,0,5,7
72,72,3,1,AL,AL,1,4,27,3,1,0,0,0,1,3,2,27,4,1,0,0,0,1,3,0,8,0,3,2
94,94,-2,0,AL,AL,7,5,45,12,2,0,2,0,0,3,5,41,5,1,0,2,0,0,3,6,10,0,2,8
107,107,-2,0,AL,AL,3,1,33,7,0,0,1,0,0,8,1,35,10,1,0,0,0,0,2,3,10,0,9,6
73260,124,-12,0,AL,AL,13,1,32,11,2,0,2,2,1,10,0,34,6,2,0,0,0,0,0,13,8,0,4,7


# add time series features

In [126]:
# create groupby object over last n games
tl_groupby = tl.groupby('team_id', as_index=False)

# run differential over last n games
rd_window_sizes = [10, 20, 30, 50, 100]
for i in rd_window_sizes:
    tl['RD_' + str(i)] = tl_groupby['RD'].shift().rolling(i).mean().bfill()

# win percentage over last n games
win_pct_window_sizes = [10, 20, 30, 50, 100]
for i in win_pct_window_sizes:
    tl['win_pct_' + str(i)] = tl_groupby['W'].shift().rolling(i).mean().bfill()

# stats over last n games
avg_window_sizes = [5, 10, 20, 30, 50, 100]
for i in avg_window_sizes:
    # offensive stats
    off_h    = tl_groupby['team_H'].shift().rolling(i).sum()['team_H']
    off_ab   = tl_groupby['team_AB'].shift().rolling(i).sum()['team_AB']
    off_bb   = tl_groupby['team_BB'].shift().rolling(i).sum()['team_BB']
    off_hbp  = tl_groupby['team_HBP'].shift().rolling(i).sum()['team_HBP']
    off_sf   = tl_groupby['team_SF'].shift().rolling(i).sum()['team_SF']
    off_1b   = tl_groupby['team_1B'].shift().rolling(i).sum()['team_1B']
    off_2b   = tl_groupby['team_2B'].shift().rolling(i).sum()['team_2B']
    off_3b   = tl_groupby['team_3B'].shift().rolling(i).sum()['team_3B']
    off_hr   = tl_groupby['team_HR'].shift().rolling(i).sum()['team_HR']
    
    # avg = H / AB
    tl['off_avg_' + str(i)] = (off_h / off_ab).bfill() 
    # OBP = (H + BB + HBP) / (AB + BB + HBP + SF)
    tl['off_obp_' + str(i)] = ((off_h + off_bb + off_hbp) / (off_ab + off_bb + off_hbp + off_sf)).bfill()
    # SLG = (1B + 2*2B + 3*3B + 4*HR) / AB
    tl['off_slg_' + str(i)] = ((off_1b + 2 * off_2b + 3 * off_3b + 4 * off_hr) / off_ab).bfill()
    # OPS = OBP + SLG
    tl['off_slg_' + str(i)] = (tl['off_obp_' + str(i)] + tl['off_slg_' + str(i)]).bfill()

    # defensive stats
    def_h = tl_groupby['opp_H'].shift().rolling(i).sum()['opp_H']
    def_ab = tl_groupby['opp_AB'].shift().rolling(i).sum()['opp_AB']
    def_bb = tl_groupby['opp_BB'].shift().rolling(i).sum()['opp_BB']
    def_ip = def_ab / 3.0
    def_er = tl_groupby['team_ER'].shift().rolling(i).sum()['team_ER']

    # avg = H / AB
    tl['def_avg_' + str(i)] = (def_h / def_ab).bfill()
    # WHIP = (W + H) / IP
    tl['def_whip_' + str(i)] = ((def_bb + def_h)/ def_ip).bfill()
    # ERA = 9 * ER / IP
    tl['def_era_' + str(i)] = (9 * def_er / def_ip).bfill()

tl.head(15)

,game_id,RD,W,opp_L,team_L,opp_R,team_R,opp_AB,opp_H,opp_2B,opp_3B,opp_HR,opp_SF,opp_HBP,opp_BB,opp_ER,team_AB,team_H,team_2B,team_3B,team_HR,team_SF,team_HBP,team_BB,team_ER,opp_id,team_id,team_1B,opp_1B,RD_10,RD_20,RD_30,RD_50,RD_100,win_pct_10,win_pct_20,win_pct_30,win_pct_50,win_pct_100,off_avg_5,off_obp_5,off_slg_5,def_avg_5,def_whip_5,def_era_5,off_avg_10,off_obp_10,off_slg_10,def_avg_10,def_whip_10,def_era_10,off_avg_20,off_obp_20,off_slg_20,def_avg_20,def_whip_20,def_era_20,off_avg_30,off_obp_30,off_slg_30,def_avg_30,def_whip_30,def_era_30,off_avg_50,off_obp_50,off_slg_50,def_avg_50,def_whip_50,def_era_50,off_avg_100,off_obp_100,off_slg_100,def_avg_100,def_whip_100,def_era_100
1,1,-3,0,AL,AL,7,4,40,15,2,0,1,0,0,1,2,31,5,0,0,1,1,0,5,7,11,0,4,12,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
15,15,7,1,AL,AL,0,7,30,4,2,0,0,0,0,1,7,36,12,1,0,3,1,0,4,0,11,0,8,2,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
25,25,1,1,AL,AL,0,1,25,0,0,0,0,0,0,4,1,30,7,3,0,0,0,0,6,0,11,0,4,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
35,35,2,1,AL,AL,1,3,29,6,1,0,0,1,0,6,0,30,6,1,0,0,0,0,0,1,8,0,5,5,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
47,47,-3,0,AL,AL,7,4,33,11,3,0,0,2,1,6,3,35,9,2,0,1,0,0,2,2,8,0,6,8,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
58,58,2,1,AL,AL,5,7,45,12,3,1,1,0,0,5,7,44,11,3,0,3,0,0,5,4,8,0,5,7,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
72,72,3,1,AL,AL,1,4,27,3,1,0,0,0,1,3,2,27,4,1,0,0,0,1,3,0,8,0,3,2,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.257143,0.321244,0.755529,0.203704,1.018519,1.166667,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
94,94,-2,0,AL,AL,7,5,45,12,2,0,2,0,0,3,5,41,5,1,0,2,0,0,3,6,10,0,2,8,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.222892,0.295082,0.650504,0.201258,1.0

In [127]:
tl[tl['game_id'] == 0]

,game_id,RD,W,opp_L,team_L,opp_R,team_R,opp_AB,opp_H,opp_2B,opp_3B,opp_HR,opp_SF,opp_HBP,opp_BB,opp_ER,team_AB,team_H,team_2B,team_3B,team_HR,team_SF,team_HBP,team_BB,team_ER,opp_id,team_id,team_1B,opp_1B,RD_10,RD_20,RD_30,RD_50,RD_100,win_pct_10,win_pct_20,win_pct_30,win_pct_50,win_pct_100,off_avg_5,off_obp_5,off_slg_5,def_avg_5,def_whip_5,def_era_5,off_avg_10,off_obp_10,off_slg_10,def_avg_10,def_whip_10,def_era_10,off_avg_20,off_obp_20,off_slg_20,def_avg_20,def_whip_20,def_era_20,off_avg_30,off_obp_30,off_slg_30,def_avg_30,def_whip_30,def_era_30,off_avg_50,off_obp_50,off_slg_50,def_avg_50,def_whip_50,def_era_50,off_avg_100,off_obp_100,off_slg_100,def_avg_100,def_whip_100,def_era_100
0,0,3,1,AL,AL,2,5,33,5,1,0,0,1,0,4,2,33,8,4,0,0,0,0,4,2,5,2,4,4,-2.3,0.2,0.0,-0.04,-0.36,0.5,0.6,0.566667,0.54,0.52,0.322034,0.396040,0.842367,0.257310,1.140351,3.157895,0.253049,0.338667,0.683179,0.284091,1.159091,3.988636,0.272727,0.360208,0.746199,0.243004,1.011782,2.902798,0.265672,0.346526,0.724635,0.249008,1.056548,3.026786,0.268524,0.346982,0.731688,0.258886,1.059242,3.103081,0.268532,0.342600,0.730215,0.265841,1.071618,3.055703
73136,0,-3,0,AL,AL,5,2,33,8,4,0,0,0,0,4,2,33,5,1,0,0,1,0,4,2,2,5,4,4,0.0,-0.9,-1.1,-0.40,-0.14,0.4,0.4,0.366667,0.42,0.48,0.263158,0.345178,0.696055,0.301775,1.313609,3.514793,0.281899,0.358442,0.744198,0.292035,1.247788,3.185841,0.272069,0.344031,0.742005,0.297376,1.298834,3.896501,0.270192,0.330427,0.756389,0.283317,1.243830,4.051333,0.265889,0.337312,0.755971,0.271598,1.180473,3.642604,0.258667,0.336914,0.747580,0.256410,1.136852,3.485689


In [128]:
# drop columns from tl that aren't features
tl_drop_cols = ['RD', 'W'] + [c for c in tl.columns if c.startswith('team') and c != 'team_id']
tl_drop_cols += [c for c in tl.columns if c.startswith('opp')]
tl_features = tl.drop(columns=tl_drop_cols)

tl_features.head()

,game_id,team_id,RD_10,RD_20,RD_30,RD_50,RD_100,win_pct_10,win_pct_20,win_pct_30,win_pct_50,win_pct_100,off_avg_5,off_obp_5,off_slg_5,def_avg_5,def_whip_5,def_era_5,off_avg_10,off_obp_10,off_slg_10,def_avg_10,def_whip_10,def_era_10,off_avg_20,off_obp_20,off_slg_20,def_avg_20,def_whip_20,def_era_20,off_avg_30,off_obp_30,off_slg_30,def_avg_30,def_whip_30,def_era_30,off_avg_50,off_obp_50,off_slg_50,def_avg_50,def_whip_50,def_era_50,off_avg_100,off_obp_100,off_slg_100,def_avg_100,def_whip_100,def_era_100
1,1,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
15,15,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
25,25,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
35,35,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961
47,47,0,-0.7,-0.45,-0.933333,-0.42,-0.5,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.6639,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.31532,0.700428,0.265951,1.091937,3.006961


# game log features and labels

In [129]:
# drop columns from gl that aren't features or labels
gl_fl_cols = [
    'game_id',
    'date',
    'visitor_id',
    'home_id',
    'visitor_L',
    'home_L',
    'date_index',
    'RD',
    'W',
    'interleague'
]

gl_features_labels = gl[gl_fl_cols].copy()

gl_features_labels.head()

,game_id,date,visitor_id,home_id,visitor_L,home_L,date_index,RD,W,interleague
0,0,1990-04-09,5,2,AL,AL,1.0,3,1,0
1,1,1990-04-09,11,0,AL,AL,1.0,-3,0,0
2,2,1990-04-09,23,3,AL,AL,1.0,1,1,0
3,3,1990-04-09,8,10,AL,AL,1.0,5,1,0
4,4,1990-04-09,14,13,AL,AL,1.0,2,1,0


# merge

In [130]:
# merge home team features
home_col_names = {c: 'home_' + c for c in tl_features.columns if c != 'game_id'}

features = pd.merge(
    gl_features_labels,
    tl_features,
    how='left',
    left_on=['game_id', 'home_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['team_id']).rename(columns=home_col_names)

# merge visiting team features
visitor_col_names = {c: 'visitor_' + c for c in tl_features.columns if c != 'game_id'}

features = pd.merge(
    features,
    tl_features,
    how='left',
    left_on=['game_id', 'visitor_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['team_id']).rename(columns=visitor_col_names)

features.head()

,game_id,date,visitor_id,home_id,visitor_L,home_L,date_index,RD,W,interleague,home_RD_10,home_RD_20,home_RD_30,home_RD_50,home_RD_100,home_win_pct_10,home_win_pct_20,home_win_pct_30,home_win_pct_50,home_win_pct_100,home_off_avg_5,home_off_obp_5,home_off_slg_5,home_def_avg_5,home_def_whip_5,home_def_era_5,home_off_avg_10,home_off_obp_10,home_off_slg_10,home_def_avg_10,home_def_whip_10,home_def_era_10,home_off_avg_20,home_off_obp_20,home_off_slg_20,home_def_avg_20,home_def_whip_20,home_def_era_20,home_off_avg_30,home_off_obp_30,home_off_slg_30,home_def_avg_30,home_def_whip_30,home_def_era_30,home_off_avg_50,home_off_obp_50,home_off_slg_50,home_def_avg_50,home_def_whip_50,home_def_era_50,home_off_avg_100,home_off_obp_100,home_off_slg_100,home_def_avg_100,home_def_whip_100,home_def_era_100,visitor_RD_10,visitor_RD_20,visitor_RD_30,visitor_RD_50,visitor_RD_100,visitor_win_pct_10,visitor_win_pct_20,visitor_win_pct_30,visitor_win_pct_50,visitor_win_pct_100,visitor_off_avg_5,visitor_off_obp_5,visitor_off_slg_5,visitor_def_avg_5,visitor_def_whip_5,visitor_def_era_5,visitor_off_avg_10,visitor_off_obp_10,visitor_off_slg_10,visitor_def_avg_10,visitor_def_whip_10,visitor_def_era_10,visitor_off_avg_20,visitor_off_obp_20,visitor_off_slg_20,visitor_def_avg_20,visitor_def_whip_20,visitor_def_era_20,visitor_off_avg_30,visitor_off_obp_30,visitor_off_slg_30,visitor_def_avg_30,visitor_def_whip_30,visitor_def_era_30,visitor_off_avg_50,visitor_off_obp_50,visitor_off_slg_50,visitor_def_avg_50,visitor_def_whip_50,visitor_def_era_50,visitor_off_avg_100,visitor_off_obp_100,visitor_off_slg_100,visitor_def_avg_100,visitor_def_whip_100,visitor_def_era_100
0,0,1990-04-09,5,2,AL,AL,1.0,3,1,0,-2.3,0.20,0.000000,-0.04,-0.36,0.5,0.60,0.566667,0.54,0.52,0.322034,0.396040,0.842367,0.257310,1.140351,3.157895,0.253049,0.338667,0.683179,0.284091,1.159091,3.988636,0.272727,0.360208,0.746199,0.243004,1.011782,2.902798,0.265672,0.346526,0.724635,0.249008,1.056548,3.026786,0.268524,0.346982,0.731688,0.258886,1.059242,3.103081,0.268532,0.342600,0.730215,0.265841,1.071618,3.055703,0.0,-0.90,-1.100000,-0.40,-0.14,0.4,0.40,0.366667,0.42,0.48,0.263158,0.345178,0.696055,0.301775,1.313609,3.514793,0.281899,0.358442,0.744198,0.292035,1.247788,3.185841,0.272069,0.344031,0.742005,0.297376,1.298834,3.896501,0.270192,0.330427,0.756389,0.283317,1.243830,4.051333,0.265889,0.337312,0.755971,0.271598,1.180473,3.642604,0.258667,0.336914,0.747580,0.256410,1.136852,3.485689
1,1,1990-04-09,11,0,AL,AL,1.0,-3,0,0,-0.7,-0.45,-0.933333,-0.42,-0.50,0.5,0.45,0.366667,0.48,0.48,0.240741,0.309392,0.685935,0.229299,1.031847,1.719745,0.218659,0.281915,0.631769,0.238938,1.132743,2.867257,0.231988,0.299346,0.663900,0.254438,1.122781,2.556213,0.233918,0.300088,0.662661,0.263054,1.155665,2.793103,0.252444,0.314525,0.702104,0.258348,1.096661,2.862917,0.248109,0.315320,0.700428,0.265951,1.091937,3.006961,-2.1,-1.05,-0.500000,-0.40,-0.08,0.2,0.40,0.466667,0.46,0.52,0.209877,0.250000,0.589506,0.308989,1.365169,4.095506,0.224615,0.265896,0.622819,0.274052,1.250729,3.463557,0.241481,0.296599,0.668450,0.250000,1.116071,3.174107,0.256158,0.317204,0.703411,0.246792,1.122409,3.091807,0.266162,0.326282,0.726981,0.249265,1.109347,3.190476,0.263561,0.330246,0.711732,0.238667,1.051936,2.966677
2,2,1990-04-09,23,3,AL,AL,1.0,1,1,0,-0.2,1.00,0.666667,0.68,0.38,0.5,0.65,0.633333,0.62,0.59,0.253165,0.338798,0.712216,0.222222,0.833333,2.500000,0.239264,0.307692,0.648183,0.226415,0.915094,3.056604,0.249622,0.314363,0.669885,0.208524,0.908676,2.589041,0.261044,0.330349,0.708863,0.220528,0.957317,2.716463,0.256611,0.321758,0.697359,0.209518,0.934487,2.569839,0.261743,0.318856,0.707335,0.235801,1.010705,2.850134,3.7,2.60,1.566667,0.36,-0.03,0.6,0.70,0.600000,0.52,0.45,0.303371,0.379808,0.801156,0.208861,1.063291,1.708861,0.265672,0.359296,0.735416,0.213836,0.952830,1.867925,0.268256,0.339450,0.743324,0.246696,1.022026,2.339207,0.266667,0.336538,0.737533,0.263056,1.015474,2.689555,0.258007,0.331942,0.715097,0.273249,1.067738,3.

# export

In [131]:
features.to_parquet('/content/drive/My Drive/mlb/features/features.parquet')